In [1]:
import numpy as np
from collections import Counter
from random import choices
from tqdm import tqdm

Definiendo el mapa con una matriz de adyacencia y dos matrices de costos (tiempo y dinero):<br>
Madrid = 0<br>
Barcelona = 1<br>
Paris = 2<br>
Lyon = 3<br>
London = 4<br>
Brussels = 5<br>
Frankfurt = 6<br>
Milan = 7<br>
Amsterdam = 8<br>
Cologne = 9<br>
Berlin = 10<br>
Roma = 11


In [2]:
tiempo_max = 72*60

names = ["Madrid", "Barcelona", "Paris", "Lyon", "London", "Brussels",
                "Frankfurt", "Milan", "Amsterdam", "Cologne", "Berlin", "Roma"]

adyacencia = np.asarray([
    #0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0], #Paris
    [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], #Lyon
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], #Brussels
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0], #Frankfurt
    [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1], #Milan
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], #Roma
])

costo_t = np.asarray([
    [0, 150, 225, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [150, 0, 390, 200, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [225, 390, 0, 112, 136, 82, 480, 0, 0, 0, 0, 0], #Paris
    [0, 200, 112, 0, 0, 0, 0, 176, 0, 0, 0, 0], #Lyon
    [0, 0, 136, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 82, 0, 0, 0, 0, 0, 120, 0, 0, 0], #Brussels
    [0, 0, 480, 0, 0, 0, 0, 454, 0, 120, 232, 0], #Frankfurt
    [0, 0, 0, 176, 0, 0, 454, 0, 0, 0, 0, 168], #Milan
    [0, 0, 0, 0, 0, 120, 0, 0, 0, 120, 364, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 120, 0, 120, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 232, 0, 364, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 168, 0, 0, 0, 0], #Roma
])

costo_d = np.asarray([
    [0, 98, 380, 0, 0, 0, 0, 0, 0, 0, 0, 0], #Madrid
    [98, 0, 400, 320, 0, 0, 0, 0, 0, 0, 0, 0], #Barcelona
    [380, 400, 0, 185, 98, 80, 345, 0, 0, 0, 0, 0], #Paris
    [0, 320, 185, 0, 0, 0, 0, 180, 0, 0, 0, 0], #Lyon
    [0, 0, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0], #London
    [0, 0, 80, 0, 0, 0, 0, 0, 48, 0, 0, 0], #Brussels
    [0, 0, 345, 0, 0, 0, 0, 240, 0, 40, 125, 0], #Frankfurt
    [0, 0, 0, 180, 0, 0, 240, 0, 0, 0, 0, 125], #Milan
    [0, 0, 0, 0, 0, 48, 0, 0, 0, 40, 235, 0], #Amsterdam
    [0, 0, 0, 0, 0, 0, 40, 0, 40, 0, 0, 0], #Cologne
    [0, 0, 0, 0, 0, 0, 125, 0, 235, 0, 0, 0], #Berlin
    [0, 0, 0, 0, 0, 0, 0, 125, 0, 0, 0, 0], #Roma
])

In [3]:
print(names)

['Madrid', 'Barcelona', 'Paris', 'Lyon', 'London', 'Brussels', 'Frankfurt', 'Milan', 'Amsterdam', 'Cologne', 'Berlin', 'Roma']


In [4]:
print(adyacencia)

[[0 1 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0]
 [1 1 0 1 1 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1 0 1 1 0]
 [0 0 0 1 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 1 1 0]
 [0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]]


In [5]:
print(costo_t)

[[  0 150 225   0   0   0   0   0   0   0   0   0]
 [150   0 390 200   0   0   0   0   0   0   0   0]
 [225 390   0 112 136  82 480   0   0   0   0   0]
 [  0 200 112   0   0   0   0 176   0   0   0   0]
 [  0   0 136   0   0   0   0   0   0   0   0   0]
 [  0   0  82   0   0   0   0   0 120   0   0   0]
 [  0   0 480   0   0   0   0 454   0 120 232   0]
 [  0   0   0 176   0   0 454   0   0   0   0 168]
 [  0   0   0   0   0 120   0   0   0 120 364   0]
 [  0   0   0   0   0   0 120   0 120   0   0   0]
 [  0   0   0   0   0   0 232   0 364   0   0   0]
 [  0   0   0   0   0   0   0 168   0   0   0   0]]


In [6]:
print(costo_d)

[[  0  98 380   0   0   0   0   0   0   0   0   0]
 [ 98   0 400 320   0   0   0   0   0   0   0   0]
 [380 400   0 185  98  80 345   0   0   0   0   0]
 [  0 320 185   0   0   0   0 180   0   0   0   0]
 [  0   0  98   0   0   0   0   0   0   0   0   0]
 [  0   0  80   0   0   0   0   0  48   0   0   0]
 [  0   0 345   0   0   0   0 240   0  40 125   0]
 [  0   0   0 180   0   0 240   0   0   0   0 125]
 [  0   0   0   0   0  48   0   0   0  40 235   0]
 [  0   0   0   0   0   0  40   0  40   0   0   0]
 [  0   0   0   0   0   0 125   0 235   0   0   0]
 [  0   0   0   0   0   0   0 125   0   0   0   0]]


In [7]:
def first_generation(pop_size, gen_size):
    generation = []
    for i in range(pop_size):
        generation.append(create_genome(gen_size))
    return generation
    
def next_generation(pop, pop_size, elitism_coef):
    fit = pop_fitness(pop)
    if np.all(fit == 0) or sum(fit) == 0:
        return first_generation(pop_size, gen_size)
    
    citiesno = cities(pop)
    temp = pop
    temp.sort(key = fitness, reverse = True)
    for i in range(len(fit)):
        if fit[i] != 0:
            fit[i]+= citiesno[i]
    
    nextgen = []
    for i in range(elitism_coef):
        nextgen.append(temp[i])
    
    for i in range(int(np.floor((pop_size - elitism_coef)/2))):
        pair = choices(pop, weights = fit, k = 2)
        nextgen.extend(mutate(pair[0], pair[1]))
    
    return nextgen
    
    
def create_genome(gen_size):
    genome = []
    genome.append(int(np.random.randint(city_no)))
    for i in range(gen_size):
        flag = True
        dobreak = False
        
        while flag:
            j  = int(np.random.randint(0, city_no))
            if adyacencia[genome[-1], j] == 1:
                flag = False
            
            if cities([genome])[0] == city_no:
                dobreak = True
        if dobreak: break
        genome.append(j)
    
    return genome

def solution(genome):
    solution = ""
    for i in range(len(genome)-1):
       solution += f"{names[genome[i]]} -> "
    solution+= f"{names[genome[-1]]}"
    return solution

def pop_fitness(population):
    fit = []
    for x in population:
        fit.append(fitness(x))

    totcost = 0
    for i in range(len(fit)):
        totcost += fit[i]
    
    cont = 0
    for i in range(len(fit)):
        if fit[i] != 0:
            cont+=1
    
    for i in range(len(fit)):
        if cont == 1:
            fit[i] = fit[i]/(totcost)
        elif(cont > 1):
            fit[i] = (1 - (fit[i]/totcost))/(cont-1)
    
            
    return fit

def fitness(genome):
    cost = 0
    time = 0
    for i in range(len(genome)-1):
        if genome[i] == genome[i+1]:
            break
        else:
            cost += costo_d[genome[i]][genome[i+1]]
            time += costo_t[genome[i]][genome[i+1]]
    if time > tiempo_max:
        return 0
    
    return cost

def cities(pop):
    cit = []
    for genome in pop:
        counter_object = Counter(genome)
        keys = counter_object.keys()
        cit.append(len(keys))
    return cit
    
def mutate(genome1, genome2): 
    if set(genome1).intersection(genome2) == set():
        child1 = create_genome(len(genome1))
        child2 = create_genome(len(genome2))
        return child1, child2
    
    while True:
        city = int(np.random.randint(0, city_no))        
        if (city in genome1) and (city in genome2):
            p = genome1.index(city)
            q = genome2.index(city)
            child1 = genome1[:p] + genome2[q:]
            child2 = genome2[:q] + genome1[p:]
            return child1, child2
    

In [8]:
city_no = len(adyacencia)-1
pop_size = 16
gen_size = 60
epochs = 600
elitism_coef = 2

genome = create_genome(gen_size)
solución = solution(genome)


print(genome)
print(solución)
print(type(genome[0]))
print(fitness(genome))

[4, 2, 6, 9, 8, 9, 8, 10, 6, 2, 4, 2, 3, 2, 1, 2, 3, 7, 6, 2, 6, 7, 3, 7, 3, 7, 3, 2, 3, 1, 0, 1, 3, 2, 5]
London -> Paris -> Frankfurt -> Cologne -> Amsterdam -> Cologne -> Amsterdam -> Berlin -> Frankfurt -> Paris -> London -> Paris -> Lyon -> Paris -> Barcelona -> Paris -> Lyon -> Milan -> Frankfurt -> Paris -> Frankfurt -> Milan -> Lyon -> Milan -> Lyon -> Milan -> Lyon -> Paris -> Lyon -> Barcelona -> Madrid -> Barcelona -> Lyon -> Paris -> Brussels
<class 'int'>
0


Creando "hijos" de la población anterior

In [9]:
genome1 = create_genome(gen_size)
genome2 = create_genome(gen_size)

children = mutate(genome1, genome2)

for child in children:
    print(child)
    print(fitness(child))
    print(solution(child))
    print()

[9, 8, 9, 8, 5, 8, 10, 8, 10, 6, 9, 6, 2, 1, 2, 5, 2, 5, 8, 9, 6, 7, 3, 7, 3, 2, 6, 10, 8, 10, 8, 5, 8, 9, 8, 5, 8, 5, 8, 10, 6, 9, 6, 10, 6, 10, 8, 10, 8, 5, 2, 5, 8, 10, 6, 9, 6, 9, 6, 2, 1, 0, 2, 6, 9, 8, 9, 8, 5, 2, 6, 9, 8, 9, 6, 10, 8, 10, 8, 9, 6]
0
Cologne -> Amsterdam -> Cologne -> Amsterdam -> Brussels -> Amsterdam -> Berlin -> Amsterdam -> Berlin -> Frankfurt -> Cologne -> Frankfurt -> Paris -> Barcelona -> Paris -> Brussels -> Paris -> Brussels -> Amsterdam -> Cologne -> Frankfurt -> Milan -> Lyon -> Milan -> Lyon -> Paris -> Frankfurt -> Berlin -> Amsterdam -> Berlin -> Amsterdam -> Brussels -> Amsterdam -> Cologne -> Amsterdam -> Brussels -> Amsterdam -> Brussels -> Amsterdam -> Berlin -> Frankfurt -> Cologne -> Frankfurt -> Berlin -> Frankfurt -> Berlin -> Amsterdam -> Berlin -> Amsterdam -> Brussels -> Paris -> Brussels -> Amsterdam -> Berlin -> Frankfurt -> Cologne -> Frankfurt -> Cologne -> Frankfurt -> Paris -> Barcelona -> Madrid -> Paris -> Frankfurt -> Cologne -> 

Evaluando la 

In [10]:
pop = first_generation(pop_size, gen_size)
pop = next_generation(pop, pop_size, elitism_coef)
fit = pop_fitness(pop)
for gene in pop:
    print(gene)
for x in fit:
    print(x)

[8, 9, 6, 10, 6, 10, 8, 10, 8, 9, 6, 2, 4, 2, 6, 2, 0, 2, 6, 7, 6, 10, 8, 5, 2, 4, 2, 3, 2, 0, 1]
[6, 7, 3, 1, 0, 2, 1, 2, 6, 10, 8, 9, 8, 10, 6, 9, 6, 10, 8, 10, 8, 9, 8, 9, 8, 10, 6, 10, 6, 9, 8, 5, 8, 9, 6, 9, 6, 10, 8, 10, 8, 5, 8, 10, 8, 10, 6, 9, 8, 9, 6, 9, 8, 5, 2, 0, 2, 6, 7, 3, 2]
[6, 2, 1, 2, 1, 2, 5, 2, 4, 2, 6, 2, 5, 8, 10, 8, 10, 6, 7, 3, 1, 2, 0, 1, 0, 2, 4, 2, 6, 9]
[7, 6, 10, 8, 9, 6, 7, 6, 2, 5, 2, 6, 7, 6, 10, 8, 5, 2, 0, 1, 3, 7, 6, 2, 4]
[8, 9, 6, 9, 8, 9, 8, 9, 6, 7, 6, 7, 6, 2, 0, 2, 3, 7, 3, 1, 0, 2, 3, 1, 3, 1, 2, 1, 0, 2, 6, 2, 1, 2, 0, 1, 0, 1, 0, 1, 3, 1, 2, 3, 1, 3, 1, 0, 1, 0, 1, 3, 1, 0, 2, 1, 2, 0, 2, 5, 2]
[1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 1, 3, 1, 0, 1, 2, 6, 7, 3, 2, 0, 1, 2, 0, 1, 2, 4, 2, 1, 2, 3, 2, 3, 7, 6, 10, 8, 5, 2, 1, 2, 0, 1, 0, 2, 1, 3, 1, 3, 7, 3, 7, 3, 7, 3, 7, 6, 7, 6, 2, 0]
[6, 7, 3, 2, 1, 2, 5, 2, 5, 2, 6, 2, 4, 2, 1, 2, 3, 1, 3, 7, 6, 2, 1, 2, 3, 7, 6, 2, 0, 1, 0, 1, 0, 2, 5, 2, 4, 2, 1, 2, 6, 7, 3, 1, 0, 1, 2, 3, 7, 3, 7, 3, 2, 5, 2, 1,

In [53]:
city_no = len(adyacencia)
pop_size = 100
gen_size = 60
epochs =1200
elitism_coef = 2

while True:
    pop = first_generation(pop_size, gen_size)
    for i in tqdm(range(epochs)):
        pop = next_generation(pop, pop_size, elitism_coef)
        if pop[-1] == pop[0]:
            break

    pop.sort(key = fitness, reverse = True)
    fit = pop_fitness(pop)
    if cities(pop)[0] == 12:
        break
    

print(solution(pop[0]))
print(cities(pop)[0])
print(fitness(pop[0]))

  0%|                                                                                | 1/1200 [00:00<00:11, 105.15it/s]

Roma -> Milan -> Lyon -> Milan -> Frankfurt -> Berlin -> Amsterdam -> Brussels -> Paris -> Barcelona -> Madrid -> Paris -> London -> Paris -> Frankfurt -> Cologne
12
2672
